In [1]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import time
from sumo_rl import SumoEnvironment
import torch

In [2]:
print("SUMO HOME:", os.environ.get("SUMO_HOME"))

SUMO HOME: /opt/miniconda3/lib/python3.13/site-packages/sumo


In [6]:
# ===== Check the actions and the state space =====
# print("Action Space:", env.action_space)
# print("State Space:", env.observation_space)

In [1]:
# ========= Simple simulation test of  SUMO-RL ===========

demo_trigger = input("Do you want to run the demo?")
if demo_trigger.lower() in ['y', 'yes']:
    print("Demo Starting....")
    exit()

    # File paths
    NET_FILE = 'single-intersection.net.xml'      # network file
    ROU_FILE = 'single-intersection-vertical.rou.xml'  # route file (traffic flows)

    # Initialize SUMO environment
    env = SumoEnvironment(
        net_file=NET_FILE,
        route_file=ROU_FILE,
        use_gui=True,                 # show SUMO GUI
        single_agent=True,            # one intersection controller
        reward_fn='diff-waiting-time',
        out_csv_name='outputs/traffic_log'
    )

    # Intialization
    obs, info = env.reset()
    avg_speed_per_road, avg_wait_time_per_road = np.zeros(4), np.zeros(4)
    avg_pressure, avg_total_speed = [],[]
    rewards, actions = [], []
    sim_time = 0.0

    for step in range(100):

        print(f"--- Step {step} / Time {sim_time:.1f} ---")

        # Take a random action
        action = env.action_space.sample()
        actions.append(action)

        # Recieve the env feedback
        obs, reward, done, truncated, info = env.step(action)

        # Current simulation time
        sim_time = env.sumo.simulation.getTime()

        # Get the vehicles IDs
        veh_ids = env.sumo.vehicle.getIDList()

        # Collisions Detection
        collisions = env.sumo.simulation.getCollidingVehiclesIDList()
        if len(collisions) > 0:
            print(f"Collisions: {collisions}")

            # Reset env after collision
            obs, info = env.reset()
            print("Environment reset.\n")
            break

        # Integrate information from the environment
        lane_counter = {}
        total_speed = 0.0
        intsec_pressure = 0.0

        for vid in veh_ids:
            speed = env.sumo.vehicle.getSpeed(vid)
            lane = env.sumo.vehicle.getLaneID(vid)
            # pos = env.sumo.vehicle.getPosition(vid)
            # print(f"{vid}: {speed:.2f} m/s on {edge} ({lane}), pos={pos}")
            total_speed += speed
            lane_counter[lane] =  lane_counter.get(lane,0) + 1
        
        avg_speed = total_speed/len(veh_ids) if len(veh_ids) > 0 else 0
        avg_total_speed.append(avg_speed)

        print(f"Average speed of all vehicles: {avg_speed:.2f} m/s")
        print(f"Vechicles per lane:{lane_counter}")


        # Arrived vehicles
        # arrived = env.sumo.simulation.getArrivedIDList()
        # if len(arrived) > 0:
        #     print(f"Arrived: {arrived}")

        # Red-Green Light States
        # for tls in env.ts_ids:
        state = env.sumo.trafficlight.getRedYellowGreenState(env.ts_ids[0])
        phase = env.sumo.trafficlight.getPhase(env.ts_ids[0])
        print(f"TraLight state=:{state} ")

        # Reward
        print(f"Step reward: {reward:.3f}\n")

        time.sleep(0.2)

        if done:
            obs, info = env.reset()
            print("Environment reset.\n")

    env.close()
    print("Demo Simulation finished!")
else:
    print("Demo skipped.")

Demo skipped.


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# File paths
NET_FILE = 'single-intersection.net.xml'      # network file
ROU_FILE = 'single-intersection-vertical.rou.xml'  # route file (traffic flows)

# Initialize SUMO environment
env = SumoEnvironment(
    net_file=NET_FILE,
    route_file=ROU_FILE,
    use_gui=True,                 # show SUMO GUI
    single_agent=True,            # one intersection controller
    reward_fn='diff-waiting-time',
    out_csv_name='outputs/traffic_log'
)

train_ppo()